In [4]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import os

book = epub.read_epub('Books/Harry_Potter_i_Wiezien_Azkabanu.epub')
text_content = ""
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        content = item.get_content()
        # Decode the content to a string
        html_content = content.decode('utf-8')

        # Use BeautifulSoup to parse the HTML and extract text
        soup = BeautifulSoup(html_content, 'lxml')

        # Find all text within the HTML body (excluding scripts and styles)
        # You might need to adjust this based on the EPUB's structure
        text = soup.get_text(separator='\n', strip=True)
        text_content += text + "\n" # Add a newline for separation between sections

print(text_content[:500])


===Y1IFVboiCSRVPEq4yQVMAg==
ROZDZIAŁ PIERWSZY
Dom Riddle’ów
M
ieszkańcy wioski Little Hangleton nadal nazywali go „Domem Riddle’ów”, choć upłynęło już wiele lat, odkąd mieszkała w nim rodzina o tym nazwisku. Stał na wzgórzu ponad wioską. Część okien zabito deskami, w dachu ziały dziury, a na fasadzie rozplenił się bluszcz. Kiedyś piękny dwór, największy i najbardziej okazały budynek w całej okolicy – teraz był ponurą, opuszczoną ruiną.
Mieszkańcy Little Hangleton byli przekonani, że w tym starym


/Users/santiagolopez/miniforge3/lib/python3.10/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/santiagolopez/miniforge3/lib/python3.10/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):
/var/folders/j2/3_89d0w54p3gxvxn4xxzh_qw0000gn/T/ipykernel_62997/681178638.py:15: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse th

In [35]:
from collections import OrderedDict, Counter
def extract_text_per_chapter(epub_path):
    """
    Extracts text content from each document item (chapter/section) in an EPUB file.

    Args:
        epub_path (str): The path to the EPUB file.

    Returns:
        OrderedDict or None: An OrderedDict where keys are item names/ids
                             and values are the extracted text for that item,
                             or None if an error occurs.
    """
    chapter_texts = OrderedDict() # Use OrderedDict to keep items in order

    try:
        # Read the EPUB book
        book = epub.read_epub(epub_path)

        # Iterate through all items in the book
        for item in book.get_items():
            # We are interested in document items (usually HTML files containing text)
            if item.get_type() == ebooklib.ITEM_DOCUMENT:
                # Get a name or identifier for the item (e.g., file name)
                item_name = item.get_name() # Or item.get_id()
                if "chapter" in item_name[:7].lower():
                    # Get the content of the item (usually bytes)
                    content = item.get_content()

                    # Decode the content to a string (assuming UTF-8 encoding)
                    # You might need to adjust encoding if necessary
                    html_content = content.decode('utf-8')

                    # Use BeautifulSoup to parse the HTML content
                    # 'lxml' parser is generally fast and robust
                    soup = BeautifulSoup(html_content, 'lxml')

                    # Extract text, removing HTML tags.
                    # get_text() concatenates text from all tags,
                    # separator='\n' adds a newline between elements,
                    # strip=True removes leading/trailing whitespace.
                    text = soup.get_text(separator=' ', strip=True)

                    # Store the extracted text with the item name as the key
                    chapter_texts[item_name] = text

    except FileNotFoundError:
        print(f"Error: EPUB file not found at '{epub_path}'")
        return None
    except Exception as e:
        print(f"Error processing EPUB file: {e}")
        return None

    return chapter_texts



In [36]:
chapters_text = extract_text_per_chapter('Books/Harry_Potter_i_Wiezien_Azkabanu.epub')
chapters_text["chapter1_8.xhtml"]

/var/folders/j2/3_89d0w54p3gxvxn4xxzh_qw0000gn/T/ipykernel_62997/2994684685.py:36: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_content, 'lxml')


'ROZDZIAŁ PIERWSZY Sowia poczta H arry Potter był chłopcem niezwykłym. Wystarczy powiedzieć, że ze wszystkich pór roku najbardziej nienawidził letnich wakacji, a\xa0można też dodać, że naprawdę chciał odrobić wszystkie prace domowe zadane na lato, ale musiał się tym zajmować w\xa0tajemnicy, kiedy wszyscy już spali. No i\xa0był czarodziejem. Dochodziła północ, a\xa0Harry leżał na brzuchu w\xa0swoim łóżku, pod kocami naciągniętymi na głowę jak namiot, z\xa0latarką w\xa0ręku i\xa0wielką, oprawioną w\xa0skórę księgą ( Dzieje magii Bathildy Bagshot) opartą o\xa0poduszkę. W\xa0drugiej ręce trzymał swoje orle pióro, którego koniuszkiem wodził po tekście, szukając czegoś, co pomogłoby mu w\xa0napisaniu rozprawki na temat: „Palenie czarownic w\xa0XIV wieku było całkowicie bezsensowne”. Pióro zatrzymało się na początku akapitu, który wzbudził jego zainteresowanie. Poprawił okulary, które opadły mu na koniec nosa, zbliżył latarkę do księgi i\xa0przeczytał: Ludzie niemagiczni (znani bardziej jako 

In [37]:
import re
import string

def tokenize_text(text):
    """
    Tokenizes text into words, removing punctuation and converting to lowercase.

    Args:
        text (str): The input text content.

    Returns:
        list: A list of words extracted from the text.
    """
    if not text:
        return []
    
    text = text.translate(str.maketrans('', '', string.punctuation))

    text = text.lower()

    # Split text into words using regular expression to handle various whitespace
    # \s+ matches one or more whitespace characters (space, tab, newline, etc.)
    words = re.split(r'\s+', text)

    words = [word for word in words if word]
    return words

In [ ]:

chapters_text = extract_text_per_chapter('Books/Harry_Potter_i_Wiezien_Azkabanu.epub')
chapters_text["chapter1_8.xhtml"]
chapter_words = tokenize_text(chapters_text["chapter1_8.xhtml"])
all_words= []
chapter_word_counts = Counter()
for item_name, text in chapters_text.items():

    chapter_words = tokenize_text(text)
    chapter_word_counts[item_name] = Counter(chapter_words)
    all_words += chapter_words
    
word_counts = Counter(all_words)
#print(word_counts.most_common(50))  # Print the 10 most common words
chapter_word_counts


/var/folders/j2/3_89d0w54p3gxvxn4xxzh_qw0000gn/T/ipykernel_62997/2994684685.py:36: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_content, 'lxml')


Counter({'chapter1_8.xhtml': Counter({'w': 95,
                  'i': 93,
                  'się': 77,
                  'na': 73,
                  'z': 62,
                  'że': 50,
                  'harry': 47,
                  'nie': 46,
                  'do': 44,
                  '–': 33,
                  'od': 28,
                  'to': 23,
                  'a': 20,
                  'był': 19,
                  'po': 19,
                  'jak': 17,
                  'mu': 17,
                  'o': 16,
                  'jego': 15,
                  'ale': 14,
                  'już': 14,
                  'było': 14,
                  'kiedy': 13,
                  'harry’ego': 13,
                  'przez': 13,
                  'tym': 12,
                  'pod': 12,
                  'ron': 12,
                  'tak': 11,
                  'bardzo': 11,
                  'co': 10,
                  'wuj': 10,
                  'hogwartu': 10,
                  'by